Copyright **`(c)`** 2023 Angelo Iannielli s317887 `<angelo.iannielli@studenti.polito.it>`  
[`https://github.com/AngeloIannielli/polito-computational-intelligence-23`]  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [193]:
from random import choices
from random import random, randint, sample
from collections import namedtuple
from copy import deepcopy, copy 
from typing import List

import lab9_lib

In [194]:
# GLOBAL VARIABLES INITIALIZATION

# problem = lab9_lib.make_problem(1)
LOCI = 1000
population = []

# taboo_list: List[Individual] = []
THRESHOLDS = 3
# DIM_FIRST_POP = 10 ** THRESHOLDS
DIM_FIRST_POP = 100

GENERATIONS = 10000
gen = 1

problem = lab9_lib.make_problem(1)

In [195]:
class Individual:
    def __init__(self):
        self.genome = []
        self.fitness = 0

    def generate_randomly(self):
        self.genome = choices([0, 1], k=LOCI)
        self.fitness = problem(self.genome)

    def generate_by_recombination(self, mut_rate, parents):
        
        #for index in range(LOCI):
        #    selected_parent = choices(parents, weights=[p.fitness for p in parents], k=1)
        #    gene = selected_parent[0].genome[index]
        #    self.genome.append(gene)

        # Define the dimension of the slices in the xover
        dim_slice = randint(1, LOCI // len(parents))

        for slice in range(LOCI // dim_slice):
            selected_parent = choices(parents, weights=[p.fitness for p in parents], k=1)
            genes = selected_parent[0].genome[slice * dim_slice : slice * dim_slice + dim_slice]
            self.genome.extend(genes)

        # Fill the genes out of the last slice
        selected_parent = choices(parents, weights=[p.fitness for p in parents], k=1)
        genes = selected_parent[0].genome[LOCI // dim_slice * dim_slice : LOCI]
        self.genome.extend(genes)

        for index in range(len(self.genome)):
            if random() < mut_rate :
                self.genome[index] = 1 if self.genome[index] == 0 else 0

    def estimate_fitness(self):
        self.fitness = problem(self.genome)

    def __str__(self):
        return f"{self.fitness:.2%} : {''.join(str(g) for g in self.genome)}"



In [196]:
class Island:
    def __init__(self, min_fitness, max_fitness, population, num_parents, mut_rate, dim_population, dim_tournament, level):
        self.min_fitness = min_fitness
        self.max_fitness = max_fitness
        self.population = population
        self.num_parents = num_parents
        self.mut_rate = mut_rate
        self.dim_population = dim_population
        self.dim_tournament = dim_tournament
        self.level = level

    def parents_selection(self):
        # sorted_population = sorted(self.population, key=lambda ind: ind.fitness, reverse=True)
        # return sorted_population[:self.dim_population]

        population_copy = copy(self.population)
        parents = []

        for _ in range(len(self.population)//self.dim_tournament):
            tournament = sample(population_copy, k= self.dim_tournament)
            tournament = sorted(tournament, key=lambda ind: ind.fitness, reverse=True)

            parents.append(tournament.pop(0))
            for loser in tournament:
                population_copy.remove(loser)

        return parents        
    
    def survival_selection(self):
        sorted_population = sorted(self.population, key=lambda ind: ind.fitness, reverse=True)

        travellers = namedtuple('travellers', ['weak', 'strong'])
        weak_population = []
        strong_population = []
        new_sorted_population = []
        

        for ind in sorted_population:

            if ind.fitness < self.min_fitness:
                weak_population.append(ind)

            elif ind.fitness > self.max_fitness:
                strong_population.append(ind)

            else:
                new_sorted_population.append(ind)
        
        return travellers(weak_population, strong_population)
    
    def shrink_population(self):
        sorted_population = sorted(self.population, key=lambda ind: ind.fitness, reverse=True)
        self.population = sorted_population[:self.dim_population]

    def get_top_solution(self) -> Individual :
        if len(self.population) > 0:
            return self.population[0]
        else: 
            return None
    
    def __str__(self):
        return f"Island {self.min_fitness:.2%}-{self.max_fitness:.2%} > POPULATION SIZE: {len(self.population)}"

In [197]:
archipelago: List[Island] = []
solution = Individual()

In [198]:
archipelago: List[Island] = []
# taboo_list: List[Individual] = []
THRESHOLDS = 3
# DIM_FIRST_POP = 10 ** THRESHOLDS
DIM_FIRST_POP = 80

solution = Individual()

for level in range(THRESHOLDS):
    archipelago.append(Island(min_fitness= 1 / THRESHOLDS * level, max_fitness= 1 / THRESHOLDS * ( level + 1), population= [], num_parents= THRESHOLDS - level + 1, mut_rate= 1 / (10 ** (level + 1)), dim_population= DIM_FIRST_POP // ( level + 1 ), dim_tournament= THRESHOLDS + 2, level= level))

# FIRST GENERATION
for _ in range(DIM_FIRST_POP):
    ind = Individual()
    ind.generate_randomly()
    archipelago[0].population.append(ind)
    
    #taboo_list.append(ind)

weak, strong = archipelago[0].survival_selection()

print("Weak dim: ", len(weak), " Strong dim: ", len(strong))

for ind in weak:
    for island in archipelago:
        if ind.fitness > island.min_fitness and ind.fitness < island.max_fitness:
            island.population.append(ind)

for ind in strong:
    for island in archipelago:
        if ind.fitness > island.min_fitness and ind.fitness < island.max_fitness:
            island.population.append(ind)

for island in archipelago:
    # Shrink the population to dim_population elements
    island.shrink_population()
    print(island)
    for ind in island.population:
        print(ind)

# Save the top solution 
for island in reversed(archipelago):
    current_top_solution = island.get_top_solution()
    if current_top_solution:
        print("Top solution:", current_top_solution)
        solution = current_top_solution
        break   

print(problem.calls)



Weak dim:  0  Strong dim:  80
Island 0.00%-33.33% > POPULATION SIZE: 80
53.20% : 1111111110011100110100110101010001001110110111111011111011111000111001000100101110111000011011110000011011100101110010101001100011010001010010111110010010100011110011000101010010100101000111101111111111100011001001100011101010000101011010011111111110000001101100011000010110111100000000001000011111010011011110111111001111011100101010110100010100110100101101001110111100101100000100011111101010101111111101000111101001101100101100011001111110110010011011110110001111101110001011111101110110111100110101010101000101110111000010001010011110100010011110110011010101000000011001011100010111000010111000111111110110100100110111011010010000001100001010111000000100001100001001001100000111000100110100010000000111000000111011111001011001101101011010110000000110010101111001101110010111111110110011101000010111100100101100101101010111011101111001000011111000101110011110110011110001111100010100011011111100011000011001110011100

In [199]:
GENERATIONS = 10000
gen = 1

# Iterate until a perfect solution is found or the number of generations is reached
while solution.fitness < 1.0 and gen < GENERATIONS:
    
    print("GENERATION", gen)

    # For each island, select a group of parents and generate the offsprings
    for island in archipelago:
        selected_parents = island.parents_selection()

        # The number of new offsprings is enough to fill the expected space of the island
        for _ in range(island.dim_population):

            # Every island needs a specific number of parents to create a new individual
            if len(selected_parents) > island.num_parents:
                parents = sample(selected_parents, k= island.num_parents)

                child = Individual()
                
                # Check if the individual already exists
                #existing_ind = True
                #while existing_ind:
                #    child.generate_by_recombination(island.mut_rate, parents)
                #    existing_ind = any(ind.genome == child.genome for ind in taboo_list)
                    
                child.generate_by_recombination(island.mut_rate, parents)

                #existing_ind = next((ind for ind in taboo_list if ind.genome == child.genome), None)
                #if existing_ind:
                #    child.fitness = existing_ind.fitness
                #else:
                #    child.estimate_fitness()
                child.estimate_fitness()

                # Update population e taboo    
                island.population.append(child)
                taboo_list.append(child)

    # For each island, there is a survival selection. The final number of individuals must be <= dim_population 
    for island in archipelago:

        # Every selection updates the population inside the island and returns two lists of individuals that are promoted/demoted
        weak, strong = island.survival_selection()

        print("Island LVL", island.level, ">", "promoted:", len(strong), "demoted:", len(weak))

        for ind in weak:
            for index in range(0, island.level):
                island = archipelago[index]
                if ind.fitness > island.min_fitness and ind.fitness < island.max_fitness:
                    island.population.append(ind)

        for ind in strong:
            for index in range(island.level + 1, len(archipelago)):
                island = archipelago[index]
                if ind.fitness > island.min_fitness and ind.fitness < island.max_fitness:
                    island.population.append(ind)

    for island in archipelago:
            # Shrink the population to dim_population elements
            island.shrink_population()
            
            print(island)
            # for ind in island.population:
                # print(ind)

    # Check the top solution 
    current_top_solution = None

    # Start by searching from the highest level
    for island in reversed(archipelago):
        current_top_solution = island.get_top_solution()
        
        # When a solution is found, it must be the better (because it is in the first position of the top level)
        if current_top_solution:
            print("Current top sol:", current_top_solution)

            # Check if the solution is better than the latest one
            if current_top_solution.fitness > solution.fitness :
                gain = (current_top_solution.fitness - solution.fitness)
                #print("Found a better solution", "(+", gain, "%):", current_top_solution)
                print(f"Found a better solution (+{gain:.2%}): {current_top_solution}")
                solution = current_top_solution
            break    

    print("Fitness called", problem.calls, "times")
    print("")            
                    
    gen += 1

GENERATION 1
Island LVL 0 > promoted: 160 demoted: 0
Island LVL 1 > promoted: 0 demoted: 0
Island LVL 2 > promoted: 0 demoted: 0
Island 0.00%-33.33% > POPULATION SIZE: 80
Island 33.33%-66.67% > POPULATION SIZE: 40
Island 66.67%-100.00% > POPULATION SIZE: 0
Current top sol: 55.80% : 111011001111111110101101111111100111111000010011010101110101100001010001100111011010110011111010101111011111011101101101000001010110011010011001111110100000110111010110100110110011101110101110011000100101110011110111010010001010011110111100110011111000000110110111111111011100010010001110100011000001110101000100011111011101110010011010011111101000101110010101111101010001101111010011110111101110010101110010110010101011111001001001100110011100110011101111001111001101000101001001011010111100100101111110011011111010011001111000110111010011011100001011010000100110100001010001101110011100000001001100011111000000010101010111101100001010101001111001011111110011111000010001001101001111000100000001011011111001001111100

Island LVL 0 > promoted: 160 demoted: 0
Island LVL 1 > promoted: 0 demoted: 0
Island LVL 2 > promoted: 0 demoted: 0
Island 0.00%-33.33% > POPULATION SIZE: 80
Island 33.33%-66.67% > POPULATION SIZE: 40
Island 66.67%-100.00% > POPULATION SIZE: 0
Current top sol: 57.40% : 0110110011111111101011011111111001111110000101110001011101011000011100011001110110101100111110101011111100001001011111001000010101100110100110011111101000001011110011001001110111110101110111111101111101011100111101110100100010100111101111001100111110000001111110010010111110111100011011111010101101011101010001000111110111111100101110100111111000001000100001110010010100111001101011011100101101111111001000111111011011001100111001001110100111111111011111110001101111100111001011110110101111001001011111100110111110100110011110001101110100110111000010110010010101110111111111100011010101000100100100110111110000000101010100111011000010101010101110100101111110010101100100111010010000111100110110100100001010110001110101110111001010